# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.copy()
df = df.groupby(["CustomerID","ProductName"])["Quantity"].sum().to_frame()
df.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pt=pd.pivot_table(df,values="Quantity",index=["ProductName"],columns=["CustomerID"]).fillna(0)
pt.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
met=pdist(pt.T,'euclidean')
met_sq=squareform(met)
distances=pd.DataFrame(1/(1+met_sq),index=pt.columns,columns=pt.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similarities=distances[distances.columns[0]].sort_values(ascending=False)[1:6]
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
df.head()
df_new=df.loc[list(similarities.index),:]
df_new.head()

Quantity
CustomerID ProductName                         
264        Apricots - Halves                  1
           Apricots Fresh                     1
           Bacardi Breezer - Tropical         1
           Bagel - Plain                      1
           Banana - Leaves                    1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
p_n =df_new.groupby(["ProductName"])["Quantity"].sum().sort_values(ascending=False)
p_n.head()

ProductName
Butter - Unsalted                3
Soup - Campbells Bean Medley     3
Towels - Paper / Kraft           3
Wine - Ej Gallo Sierra Valley    3
Pepper - Black, Whole            3
Name: Quantity, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
lst=data[data["CustomerID"]== 33 ]["ProductName"].to_list()
lst2=[e for e in list(p_n.index) if e not in lst]
display(p_n.loc[lst2].head())


ProductName
Butter - Unsalted                3
Soup - Campbells Bean Medley     3
Wine - Ej Gallo Sierra Valley    3
Wine - Blue Nun Qualitatswein    3
Scallops 60/80 Iqf               2
Name: Quantity, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
def generaSimilitud(id_e):
    sim=distances[id_e].sort_values(ascending=False)[1:]
    df_new2=df.loc[list(sim.index),:]
    p_n2 =df_new2.groupby(["ProductName"])["Quantity"].sum().sort_values(ascending=False)
    lst_func=data[data["CustomerID"]== id_e ]["ProductName"].to_list()
    lst2_func=[e for e in list(p_n2.index) if e not in lst_func]
    return lst2_func[:5]
d=dict()

In [15]:

lstId=list(set(data["CustomerID"].values))
len(lstId)

1000

In [13]:
for e in lstId[:100]:
    d[e]=generaSimilitud(e)



In [14]:
for e in lstId[100:200]:
    d[e]=generaSimilitud(e)

In [16]:
for e in lstId[200:300]:
    d[e]=generaSimilitud(e)

In [17]:
for e in lstId[300:400]:
    d[e]=generaSimilitud(e)

In [18]:
for e in lstId[400:500]:
    d[e]=generaSimilitud(e)

In [19]:
for e in lstId[500:600]:
    d[e]=generaSimilitud(e)

In [20]:
for e in lstId[600:700]:
    d[e]=generaSimilitud(e)

In [21]:
for e in lstId[700:800]:
    d[e]=generaSimilitud(e)

In [22]:
for e in lstId[800:900]:
    d[e]=generaSimilitud(e)

In [23]:
for e in lstId[900:len(lstId)]:
    d[e]=generaSimilitud(e)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [30]:
d_n=dict()
prod1=[e[0] for e in list(d.values())]
ids=list(d.keys())
#d_n["CustomerID"] = list(d.keys())
#d_n[]
#df_guardar = pd.DataFrame(data=d.items(),columns=["CustomerID","Product_1","Product_2","Product_3","Product_4","Product_5"])
#df_guardar.head()

In [31]:
prod2=[e[1] for e in list(d.values())]

In [32]:
prod3=[e[2] for e in list(d.values())]

In [33]:
prod4=[e[3] for e in list(d.values())]

In [34]:
prod5=[e[4] for e in list(d.values())]

In [36]:
d_n={
    "CustomerID":ids,
    "Product_1": prod1,
    "Product_2": prod2,
    "Product_3": prod3,
    "Product_4": prod4,
    "Product_5": prod5,
    
}
df_fin=pd.DataFrame(data=d_n)
df_fin.head()

,CustomerID,Product_1,Product_2,Product_3,Product_4,Product_5
0,83973,Cookies - Assorted,Flavouring - Orange,Fenngreek Seed,Beef - Montreal Smoked Brisket,Snapple Lemon Tea
1,59399,Cookies - Assorted,Flavouring - Orange,Wine - White Cab Sauv.on,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket
2,92168,Fenngreek Seed,Wine - White Cab Sauv.on,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket,Oil - Shortening - All - Purpose
3,49159,Cookies - Assorted,Flavouring - Orange,Fenngreek Seed,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket
4,18441,Flavouring - Orange,Fenngreek Seed,Wine - White Cab Sauv.on,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [37]:
met2=pdist(pt.T,'correlation')
met_sq2=squareform(met2)
distances2=pd.DataFrame(1/(1+met_sq2),index=pt.columns,columns=pt.columns)

def generaSimilitud2(id_e):
    sim=distances2[id_e].sort_values(ascending=False)[1:]
    df_new2=df.loc[list(sim.index),:]
    p_n2 =df_new2.groupby(["ProductName"])["Quantity"].sum().sort_values(ascending=False)
    lst_func=data[data["CustomerID"]== id_e ]["ProductName"].to_list()
    lst2_func=[e for e in list(p_n2.index) if e not in lst_func]
    return lst2_func[:5]
d2=dict()

In [38]:
for e in lstId[:100]:
    d2[e]=generaSimilitud2(e)




In [39]:
for e in lstId[100:200]:
    d2[e]=generaSimilitud2(e)

In [40]:
for e in lstId[200:300]:
    d2[e]=generaSimilitud2(e)

In [41]:
for e in lstId[300:400]:
    d2[e]=generaSimilitud2(e)

In [42]:
for e in lstId[400:500]:
    d2[e]=generaSimilitud2(e)

In [43]:
for e in lstId[500:600]:
    d2[e]=generaSimilitud2(e)

In [44]:
for e in lstId[600:700]:
    d2[e]=generaSimilitud2(e)

In [45]:
for e in lstId[700:800]:
    d2[e]=generaSimilitud2(e)

In [46]:
for e in lstId[800:900]:
    d2[e]=generaSimilitud2(e)

In [47]:
for e in lstId[900:1000]:
    d2[e]=generaSimilitud2(e)

In [48]:
prod1_2=[e[0] for e in list(d2.values())]
ids2=list(d2.keys())
prod2_2=[e[1] for e in list(d2.values())]
prod3_2=[e[2] for e in list(d2.values())]
prod4_2=[e[3] for e in list(d2.values())]
prod5_2=[e[4] for e in list(d2.values())]

In [49]:
d_n2={
    "CustomerID":ids2,
    "Product_1": prod1_2,
    "Product_2": prod2_2,
    "Product_3": prod3_2,
    "Product_4": prod4_2,
    "Product_5": prod5_2,
    
}
df_fin2=pd.DataFrame(data=d_n2)
display(df_fin.head())
display(df_fin2.head())

,CustomerID,Product_1,Product_2,Product_3,Product_4,Product_5
0,83973,Cookies - Assorted,Flavouring - Orange,Fenngreek Seed,Beef - Montreal Smoked Brisket,Snapple Lemon Tea
1,59399,Cookies - Assorted,Flavouring - Orange,Wine - White Cab Sauv.on,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket
2,92168,Fenngreek Seed,Wine - White Cab Sauv.on,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket,Oil - Shortening - All - Purpose
3,49159,Cookies - Assorted,Flavouring - Orange,Fenngreek Seed,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket
4,18441,Flavouring - Orange,Fenngreek Seed,Wine - White Cab Sauv.on,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket


,CustomerID,Product_1,Product_2,Product_3,Product_4,Product_5
0,83973,Cookies - Assorted,Flavouring - Orange,Fenngreek Seed,Beef - Montreal Smoked Brisket,Snapple Lemon Tea
1,59399,Cookies - Assorted,Flavouring - Orange,Wine - White Cab Sauv.on,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket
2,92168,Fenngreek Seed,Wine - White Cab Sauv.on,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket,Oil - Shortening - All - Purpose
3,49159,Cookies - Assorted,Flavouring - Orange,Fenngreek Seed,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket
4,18441,Flavouring - Orange,Fenngreek Seed,Wine - White Cab Sauv.on,Bandage - Flexible Neon,Beef - Montreal Smoked Brisket
